In [1]:
# Imports:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import sklearn
from sklearn.metrics import mean_squared_error
import plotly.express as px
from scipy import stats


In [2]:
#set seed for reproducibility
random_state = 42
np.random.seed(random_state)

In [3]:
# same dataset as bidirectional, which is why the path is the same
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_norway_dataset_dict.pickle', 'rb') as f:
    cananda_data_dict = pickle.load(f)

In [4]:
norway_data = pd.read_csv(r'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/resources/norway_cpi_dataset.csv')

--------

### Horizon Analyses:

In [5]:
# import horizon files 
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - norway/data/hor1_test_dataset.pickle', 'rb') as f:
    norway_hor_1_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - norway/data/hor2_test_dataset.pickle', 'rb') as f:
    norway_hor_2_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - norway/data/hor3_test_dataset.pickle', 'rb') as f:
    norway_hor_3_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - norway/data/hor4_test_dataset.pickle', 'rb') as f:
    norway_hor_4_test_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - norway/data/hor8_test_dataset.pickle', 'rb') as f:
    norway_hor_8_test_dict = pickle.load(f)

In [6]:
norway_hor_1_test_dict['All-items'].shape

(29, 14)

In [7]:
import sys        
sys.path.append('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model_norway/model')       
from GRU_model import *
from config import * 
from utils import *
Lr = 0.04329823452489634

sys.path.append('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model')       
from pipeline_config import *

2023-09-30 23:28:42.193001: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
num_forecast_steps = 8

category_horizon_dict = {}

for category_name in list(norway_hor_1_test_dict.keys()):
    # load model
    cat_model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
    cat_optimizer = torch.optim.AdamW(cat_model.parameters(), lr=Lr)
    cat_model.to(Device)

    category_model, optimizer, checkpoint, valid_loss_min = load_checkpoint('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model_norway/models_weights/'+category_name+'.pt', cat_model, cat_optimizer)

    horizon_dict = {}

    # Generate forecasts step by step
    for step in range(num_forecast_steps):
        with torch.no_grad():
            # create dataloader
            x_test = norway_hor_1_test_dict[category_name].iloc[:,:-1].to_numpy()
            y_test= norway_hor_1_test_dict[category_name].iloc[:,-1:].to_numpy()

            x_test = torch.from_numpy(x_test).to(torch.float32)
            y_test = torch.from_numpy(y_test).to(torch.float32)

            test_dataset = TensorDataset(x_test, y_test)

            test_dataloader =  DataLoader(test_dataset, batch_size=BatchSize, shuffle=False)

            for inputs, labels in test_dataloader:
                # since each batch contains 32 samples, and the largest df is of size 32, then we dont need to concat the outputs per batch per category.
                inputs = inputs.view(inputs.shape[0], SequenceLength, Features)
                #print(f'inputs shape: {inputs.shape}')
                #print(f'labels shape: {labels.shape}')
                inputs, labels = inputs.to(Device), labels.to(Device)
                out = category_model(inputs)
                out_df = pd.DataFrame(out)
                
                first = norway_hor_1_test_dict[category_name].iloc[:,1:].reset_index(drop = True).copy()
                print(f'first shape: {first.shape}')
                second = out_df.reset_index(drop = True).rename(columns = {0: 'Inflation t+'+str(2+step)})
                print(f'second shape: {second.shape}')

                norway_hor_1_test_dict[category_name] = pd.concat([first, second],axis=1) #prediction

                horizon_dict[step+2] = norway_hor_1_test_dict[category_name]

    category_horizon_dict[category_name] = horizon_dict
    


first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second shape: (29, 1)
first shape: (29, 13)
second sha

In [9]:
def avg_rmse_hor(prediction_dict, actual_dict, horizon):
    mse_lst = []
    for key in list(actual_dict.keys()):
        predictions = prediction_dict[key][horizon].iloc[:,-1:].reset_index(drop = True)
        actuals = actual_dict[key].iloc[:,-1:].reset_index(drop = True)

        y_pred = predictions.values
        y_actual = actuals.values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        #print(f'y pred: {predicted_values_non_null}')
        #print(f'y actual: {observed_values_non_null}')
        mse = mean_squared_error(predicted_values_non_null, observed_values_non_null)
        mse_lst.append(mse)
    
    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    
    print(f'RMSE:  {avg_rmse}')
    print(f'MSE std:  {rmse_std}')
    print(f'interval: {[avg_rmse-rmse_std, avg_rmse+rmse_std]}')

    return avg_rmse,rmse_std

In [10]:
avg_rmse_hor(category_horizon_dict, norway_hor_2_test_dict, 2)

RMSE:  2.1497360622611845
MSE std:  1.8376399214349
interval: [0.31209614082628456, 3.9873759836960847]


(2.1497360622611845, 1.8376399214349)

In [11]:
avg_rmse_hor(category_horizon_dict, norway_hor_3_test_dict, 3)

RMSE:  2.217776254889977
MSE std:  1.9048352527855092
interval: [0.3129410021044676, 4.122611507675487]


(2.217776254889977, 1.9048352527855092)

In [12]:
avg_rmse_hor(category_horizon_dict, norway_hor_4_test_dict, 4)

RMSE:  2.209141784180778
MSE std:  1.9932456345444571
interval: [0.21589614963632076, 4.202387418725235]


(2.209141784180778, 1.9932456345444571)

In [13]:
avg_rmse_hor(category_horizon_dict, norway_hor_8_test_dict, 8)

RMSE:  2.4011743000505033
MSE std:  2.0759181624804453
interval: [0.325256137570058, 4.4770924625309485]


(2.4011743000505033, 2.0759181624804453)

In [14]:
#category_horizon_dict - prediction dict per category per horizon
# us_hor_x_test_dict - actual dict per category for horizon x

In [15]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_norway_2_period_dataset_dict.pickle', 'rb') as f:
    hor_2_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_norway_3_period_dataset_dict.pickle', 'rb') as f:
    hor_3_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_norway_4_period_dataset_dict.pickle', 'rb') as f:
    hor_4_raw_dataset_dict = pickle.load(f)

with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/bi_directional_norway_8_period_dataset_dict.pickle', 'rb') as f:
    hor_8_raw_dataset_dict = pickle.load(f)

In [16]:
def create_test_dataframe(raw_dataset_dict: dict, horizon:int):
    test_dict = {}
    for key in raw_dataset_dict.keys():
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Inflation t+'+str(horizon)]]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+'+str(horizon): 'Actual Horizon '+str(horizon)}, inplace=True)
        target_df = df[df['Year'] > 2020]
        target_df = target_df.reset_index(drop = True)
        test_dict[key] = target_df
    return test_dict

In [17]:
test_dict_hor_2 = create_test_dataframe(hor_2_raw_dataset_dict, 2)
test_dict_hor_3 = create_test_dataframe(hor_3_raw_dataset_dict, 3)
test_dict_hor_4 = create_test_dataframe(hor_4_raw_dataset_dict, 4)
test_dict_hor_8 = create_test_dataframe(hor_8_raw_dataset_dict, 8)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_10053/3431962343.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_10053/3431962343.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+'+str(horizon): 'Actual Horizon '+str(horizon)}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_10053/3431962343.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_10053/3431962343.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_10053/3431962343.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+'+str(horizon): 'Actual Horizon '+str(horizon)}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_10053/3431962343.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [18]:
def get_df_with_predictions_horizon(prediction_dic: dict, actual_dic:dict, horizon: int) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key][horizon].iloc[:,-1:].reset_index(drop = True)
        predictions = predictions.rename(columns = {f'Inflation t+{horizon}': f'Prediction Horizon {horizon}'})
        actuals = actual_dic[key]

        data = pd.concat([predictions, actuals], axis=1).loc[:,['Category','Date','Year',f'Prediction Horizon {horizon}', f'Actual Horizon {horizon}']]
        all_data_dict[key] = data
        
    return all_data_dict

In [19]:
all_data_test_dict_hor_2 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_2,2)
all_data_test_dict_hor_3 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_3,3)
all_data_test_dict_hor_4 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_4,4)
all_data_test_dict_hor_8 = get_df_with_predictions_horizon(category_horizon_dict, test_dict_hor_8,8)

In [20]:
all_data_test_dict_hor_2['All-items']

,Category,Date,Year,Prediction Horizon 2,Actual Horizon 2
0,All-items,2021-01-15,2021.0,-0.006464,-0.261438
1,All-items,2021-02-15,2021.0,0.069717,0.348432
2,All-items,2021-03-15,2021.0,0.288150,-0.086994
3,All-items,2021-04-15,2021.0,0.356635,0.347524
4,All-items,2021-05-15,2021.0,0.163157,0.863563
5,All-items,2021-06-15,2021.0,0.238805,0.000000
6,All-items,2021-07-15,2021.0,-0.262081,1.026527
7,All-items,2021-08-15,2021.0,0.249949,-0.255646
8,All-items,2021-09-15,2021.0,0.180810,0.764985
9,All-items,2021-10-15,2021.0,-0.180912,0.675108


In [21]:
def plot_results_horizon(all_data_dict, categories, horizon):
    #category_samples = random.sample(categories, 5)+['All items']
    category_samples = categories
    for category in category_samples:
        category_df = all_data_dict[category]
        y_pred = category_df[f'Prediction Horizon {horizon}'].values
        y_actual = category_df[f'Actual Horizon {horizon}'].values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        mse = mean_squared_error(predicted_values_non_null, observed_values_non_null)
        print(f'Category is: {category}')
        print(f'RMSE is: {np.sqrt(mse)}')

        fig = px.line(category_df, x="Date", y=[f'Actual Horizon {horizon}', f'Prediction Horizon {horizon}'], title=f'{category} - Actual VS Prediction: horizon {horizon}')
        fig.show()



In [22]:
def total_corr_horizon(all_data_test_dict, horizon):
    corr_dict = {}
    for key in list(all_data_test_dict.keys()):
        df = all_data_test_dict[key]
        y_pred = df[f'Prediction Horizon {horizon}'].values
        y_actual = df[f'Actual Horizon {horizon}'].values

        non_null_indices = ~np.isnan(y_pred) & ~np.isnan(y_actual)

        predicted_values_non_null = y_pred[non_null_indices]
        observed_values_non_null = y_actual[non_null_indices]

        corr = stats.pearsonr(predicted_values_non_null,observed_values_non_null)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    print(f'Total Corr Horizon {horizon}, {total_corr}')
    return total_corr

In [23]:
total_corr_horizon(all_data_test_dict_hor_2, 2)
total_corr_horizon(all_data_test_dict_hor_3, 3)
total_corr_horizon(all_data_test_dict_hor_4, 4)
total_corr_horizon(all_data_test_dict_hor_8, 8)

Number of categories with High Correlation: 1
Total Corr Horizon 2, -2.8154851260719878
Number of categories with High Correlation: 1
Total Corr Horizon 3, -0.9697525615384622
Number of categories with High Correlation: 1
Total Corr Horizon 4, 0.8973798968953731
Number of categories with High Correlation: 0
Total Corr Horizon 8, -2.8596176881652378


-2.8596176881652378

In [24]:
cats = list(all_data_test_dict_hor_2.keys())
categories = random.sample(cats, 5)+['All-items']

In [25]:
#categories = list(all_data_test_dict_hor_2.keys())
plot_results_horizon(all_data_test_dict_hor_2, categories, 2)

Category is: Furnishings, household equipment and routine maintenance
RMSE is: 3.07307160205279


Category is: Personal effects n.e.c.
RMSE is: 1.2367755032946226


Category is: Out-patient services
RMSE is: 0.5986253397227871


Category is: Communications
RMSE is: 0.8297471803526395


Category is: Imputed rentals for housing
RMSE is: 0.5976814764706687


Category is: All-items
RMSE is: 0.6229640535484853


In [26]:
#categories = list(all_data_test_dict_hor_3.keys())
plot_results_horizon(all_data_test_dict_hor_3, categories, 3)

Category is: Furnishings, household equipment and routine maintenance
RMSE is: 2.554343565394564


Category is: Personal effects n.e.c.
RMSE is: 1.3202966333043704


Category is: Out-patient services
RMSE is: 0.6873112831799587


Category is: Communications
RMSE is: 0.7929380518663579


Category is: Imputed rentals for housing
RMSE is: 0.6944244844716211


Category is: All-items
RMSE is: 0.6592687190700197


In [27]:
#categories = list(all_data_test_dict_hor_4.keys())
plot_results_horizon(all_data_test_dict_hor_4, categories, 4)

Category is: Furnishings, household equipment and routine maintenance
RMSE is: 2.623945957211996


Category is: Personal effects n.e.c.
RMSE is: 1.2430001788483378


Category is: Out-patient services
RMSE is: 0.7427148788596721


Category is: Communications
RMSE is: 0.8380271665073984


Category is: Imputed rentals for housing
RMSE is: 0.4863068607799816


Category is: All-items
RMSE is: 0.6798822918282155


In [28]:
#categories = list(all_data_test_dict_hor_8.keys())
plot_results_horizon(all_data_test_dict_hor_8, categories, 8)

Category is: Furnishings, household equipment and routine maintenance
RMSE is: 2.4624103622803153


Category is: Personal effects n.e.c.
RMSE is: 1.3093967567201934


Category is: Out-patient services
RMSE is: 1.148244035181207


Category is: Communications
RMSE is: 0.8870038028564154


Category is: Imputed rentals for housing
RMSE is: 0.8574021699822914


Category is: All-items
RMSE is: 0.706191053049932
